In [8]:
# Celda 1: Importaciones Necesarias
import requests
import json
import os

print("Librerías importadas correctamente.")

Librerías importadas correctamente.


In [ ]:
# Celda 2: Configuración Global
# BASE_URL = "http://localhost:8000"  # Asegúrate de que el puerto coincida
BASE_URL = "https://pdf2image.qaiaanalytics.com"  # Asegúrate de que el puerto coincida
OCR_ENDPOINT_URL = f"{BASE_URL}/convert-pdf"

# Lista de archivos PDF de prueba (colócalos en el mismo directorio que este notebook)
# Puedes añadir más nombres de archivo aquí.
PDF_FILENAMES_TO_TEST = [
    "NAN_FEHP701932.pdf",
    # "otro_documento.pdf", # Descomenta y añade más PDFs para probar
    # "documento_multiples_paginas.pdf"
]

print(f"Endpoint a probar: {OCR_ENDPOINT_URL}")
print(f"Archivos PDF que se intentarán probar: {PDF_FILENAMES_TO_TEST}")

Endpoint a probar: http://localhost:8000/convert-pdf
Archivos PDF que se intentarán probar: ['NAN_FEHP701932.pdf']


In [33]:
# Celda 3: Función para Probar el Endpoint /convert-pdf con un Archivo Específico
def probar_ocr_de_pdf(pdf_filename: str):
    """
    Sube un archivo PDF al endpoint /convert-pdf y muestra la respuesta.
    """
    print(f"\n--- Probando OCR para el archivo: {pdf_filename} ---")
    
    pdf_file_path = os.path.join(os.getcwd(), pdf_filename)
    
    if not os.path.exists(pdf_file_path):
        print(f"  ❌ ERROR: El archivo PDF '{pdf_file_path}' no fue encontrado. Omitiendo esta prueba.")
        return None # Retorna None si el archivo no existe

    print(f"  Ruta completa: {pdf_file_path}")
    
    try:
        # Abrir el archivo PDF en modo binario para la subida
        with open(pdf_file_path, 'rb') as f:
            # 'files' es un diccionario donde la clave 'file' debe coincidir
            # con el nombre del parámetro File(...) en tu endpoint FastAPI.
            # La tupla contiene: (nombre_de_archivo_original, objeto_archivo, tipo_mime)
            files_payload = {'file': (pdf_filename, f, 'application/pdf')}
            
            print(f"  Enviando solicitud POST a {OCR_ENDPOINT_URL}...")
            response_ocr = requests.post(OCR_ENDPOINT_URL, files=files_payload, timeout=60) # Timeout de 60 segundos
            
            # Verificar si la solicitud fue exitosa (código 2xx)
            response_ocr.raise_for_status() 
            
            # Obtener la respuesta JSON
            ocr_data = response_ocr.json()
            
            print(f"\n  [Respuesta de /convert-pdf para '{pdf_filename}' - Éxito (Estado {response_ocr.status_code})]")
            print(  "  -------------------------------------------------------------------")

            # 'pages' es una LISTA de diccionarios, cada uno representando una página.
            lista_de_paginas_info = ocr_data.get('pages', [])

            #  Construir el texto completo concatenando el texto de cada página
            textos_de_paginas = []
            if isinstance(lista_de_paginas_info, list): # Asegurarse de que es una lista
                for pagina_info in lista_de_paginas_info:
                    if isinstance(pagina_info, dict) and 'text' in pagina_info: # Asegurarse de que es un dict con 'text'
                        textos_de_paginas.append(pagina_info.get('text', ''))
            # Unir texto de páginas con dos saltos de línea para simular separación de páginas
            texto_completo_extraido = "\n\n".join(textos_de_paginas)
            
            #  El 'detalle_por_pagina' es directamente la lista_de_paginas_info
            detalle_por_pagina = lista_de_paginas_info
            # --- FIN DE AJUSTES ---
            
            print(f"  Texto Completo OCR (primeros 300 caracteres):\n  '{texto_completo_extraido[:300].replace('\n', '\n  ')}'...")

            
            if detalle_por_pagina:
                print(f"\n  Detalle por página (mostrando hasta 2 páginas):")
                for i, page_data in enumerate(detalle_por_pagina):
                    if i < 2: # Limitar la salida
                        print(f"    Página {page_data.get('page')}: '{page_data.get('text', '')[:100].replace('\n', ' ')}...'")
                    elif i == 2:
                        print(f"    ... y {len(detalle_por_pagina) - 2} página(s) más.")
                        break
            else:
                print("  No se encontró detalle por página o está vacío.")
            
            if texto_completo_extraido.strip():
                print("\n  ✅ Prueba de OCR Parece Exitosa: Se extrajo texto.")
            else:
                print("\n  ⚠️ Prueba de OCR: El endpoint respondió correctamente, pero no se extrajo texto significativo.")
            
            return ocr_data # Devuelve los datos para posible uso posterior

    except requests.exceptions.Timeout:
        print(f"  [ERROR DE TIMEOUT] La solicitud a {OCR_ENDPOINT_URL} para '{pdf_filename}' tardó demasiado.")
    except requests.exceptions.ConnectionError:
        print(f"  [ERROR DE CONEXIÓN] No se pudo conectar a la API en {BASE_URL}.")
        print("  Asegúrate de que el servidor FastAPI (uvicorn) esté ejecutándose.")
    except requests.exceptions.HTTPError as http_err:
        print(f"  [ERROR HTTP: {http_err.response.status_code}] Para '{pdf_filename}'. Respuesta del servidor: {http_err.response.text}")
    except json.JSONDecodeError:
        print(f"  [ERROR DE JSON] La respuesta del servidor para '{pdf_filename}' no es un JSON válido.")
        print(f"  Respuesta recibida: {response_ocr.text if 'response_ocr' in locals() and response_ocr else 'No response object'}")
    except Exception as e:
        print(f"  [ERROR INESPERADO] Ocurrió un error procesando '{pdf_filename}': {e}")
        import traceback
        traceback.print_exc()
    return None # Retorna None si hay error

In [34]:
# Celda 4: Ejecutar las Pruebas de OCR para cada Archivo PDF Definido

print("\n" + "="*10 + " INICIANDO PRUEBAS DEL ENDPOINT /convert-pdf " + "="*10)

resultados_ocr = {} # Diccionario para guardar los resultados si se quieren usar después

if not PDF_FILENAMES_TO_TEST:
    print("No se han definido archivos PDF en 'PDF_FILENAMES_TO_TEST'. Añade nombres de archivo para probar.")
else:
    for pdf_file in PDF_FILENAMES_TO_TEST:
        resultado = probar_ocr_de_pdf(pdf_file)
        if resultado:
            resultados_ocr[pdf_file] = resultado # Guardar el resultado si fue exitoso

print("\n" + "="*10 + " FIN DE LAS PRUEBAS DEL ENDPOINT /convert-pdf " + "="*10)

# Opcional: Mostrar un resumen de los textos extraídos (si se guardaron)
if resultados_ocr:
    print("\nResumen de Textos Extraídos (si alguno fue exitoso):")
    for filename, data in resultados_ocr.items():
        print(f"  Archivo: {filename} - Texto (primeros 50 chars): '{data.get('texto_completo_ocr', '')[:50].replace('\n',' ')}...'")


========== INICIANDO PRUEBAS DEL ENDPOINT /convert-pdf ==========

--- Probando OCR para el archivo: NAN_FEHP701932.pdf ---
  Ruta completa: c:\repositorios\pdf2Image\NAN_FEHP701932.pdf
  Enviando solicitud POST a http://localhost:8000/convert-pdf...

  [Respuesta de /convert-pdf para 'NAN_FEHP701932.pdf' - Éxito (Estado 200)]
  -------------------------------------------------------------------
  Texto Completo OCR (primeros 300 caracteres):
  'E.S.E. HSOPITAL SAN RAFAEL DE PACHO
  eS MACROPROCESO DE GESTION FINANCIERA FAC-FM-020
  PROCESO DE FACTURACION VERSION
  HOSPITAL FORMATO DE RECIBIDO USUARIO V01-2017
  
  Certifico con mi firma que recibi a satisfacci6én servicios de la siguiente dependencia:
  Fecha de la prestacion: 02-05-2025
  
  LABORATORIO '...

  Detalle por página (mostrando hasta 2 páginas):
    Página 1: 'E.S.E. HSOPITAL SAN RAFAEL DE PACHO eS MACROPROCESO DE GESTION FINANCIERA FAC-FM-020 PROCESO DE FACT...'

  ✅ Prueba de OCR Parece Exitosa: Se extrajo texto.

===

### Prueba Local 

In [35]:
# Celda 1: Importaciones Necesarias para la Lógica de OCR
import pytesseract
from pdf2image import convert_from_bytes, pdfinfo_from_bytes # pdfinfo_from_bytes es opcional pero útil
from PIL import Image # Pillow es usado por pdf2image y para manipulación opcional
import os
import io # No estrictamente necesario si lees bytes directamente del archivo

print("Librerías para OCR local importadas.")
# Opcional: Configurar la ruta al ejecutable de Tesseract si no está en el PATH
# try:
#     pytesseract.get_tesseract_version()
#     print(f"Tesseract OCR encontrado: Versión {pytesseract.get_tesseract_version()}")
# except pytesseract.TesseractNotFoundError:
#     print("Tesseract OCR NO encontrado. Asegúrate de que esté instalado y en el PATH.")
#     print("Si es necesario, configura la ruta manualmente con:")
#     print("# pytesseract.pytesseract.tesseract_cmd = r'/ruta/a/tesseract'") # Ejemplo para Linux/macOS
#     print("# pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'") # Ejemplo para Windows
#     # Es mejor que Tesseract esté en el PATH del sistema.

Librerías para OCR local importadas.


In [36]:
# Celda 2: Configuración del Archivo PDF de Prueba
PDF_FILENAME_FOR_DIRECT_TEST = "NAN_FEHP701932.pdf" # CAMBIA ESTO SI ES NECESARIO
pdf_file_path_direct_test = os.path.join(os.getcwd(), PDF_FILENAME_FOR_DIRECT_TEST)

if not os.path.exists(pdf_file_path_direct_test):
    print(f"¡ERROR! El archivo PDF de prueba '{pdf_file_path_direct_test}' no fue encontrado.")
    print("Por favor, crea un archivo PDF o corrige la ruta en 'PDF_FILENAME_FOR_DIRECT_TEST'.")
    pdf_existe_direct_test = False
else:
    print(f"Archivo PDF de prueba para lógica directa encontrado en: {pdf_file_path_direct_test}")
    pdf_existe_direct_test = True

Archivo PDF de prueba para lógica directa encontrado en: c:\repositorios\pdf2Image\NAN_FEHP701932.pdf


In [37]:
# Celda 3: Función que Emula la Lógica del Endpoint /convert-pdf

def procesar_pdf_localmente(pdf_path: str, dpi_setting: int = 300, tesseract_lang: str = 'spa'):
    """
    Emula la lógica de extracción de texto de un PDF tal como se haría en el endpoint.
    """
    print(f"\n--- Procesando PDF localmente: {pdf_path} ---")
    
    if not os.path.exists(pdf_path):
        print("  Error: El archivo PDF no existe en la ruta proporcionada.")
        return None

    try:
        # Leer los bytes del archivo PDF
        with open(pdf_path, 'rb') as f:
            pdf_bytes = f.read()

        # Opcional: Obtener información del PDF (número de páginas, etc.)
        # try:
        #     info = pdfinfo_from_bytes(pdf_bytes, userpw=None, poppler_path=None)
        #     print(f"  Información del PDF: {info}")
        # except Exception as e_info:
        #     print(f"  No se pudo obtener información del PDF (Poppler podría no estar configurado): {e_info}")


        print(f"  Convirtiendo PDF a imágenes (DPI: {dpi_setting})...")
        # Convierte todas las páginas del PDF a imágenes
        # poppler_path=None usará el PATH del sistema. Configúralo si es necesario.
        images = convert_from_bytes(pdf_bytes, dpi=dpi_setting, poppler_path=None) 
        
        if not images:
            print("  No se pudieron generar imágenes del PDF. El PDF podría estar vacío o corrupto.")
            return {"texto_completo_ocr": "", "detalle_por_pagina": []}

        print(f"  Se generaron {len(images)} imágenes del PDF.")

        ocr_results_per_page = []
        full_ocr_text_parts = []

        for i, image_pil in enumerate(images):
            print(f"  Procesando OCR para la página {i + 1}...")
            
            # Opcional: Pre-procesamiento de la imagen (ej. convertir a escala de grises)
            # gray_image = image_pil.convert("L")
            # text_page = pytesseract.image_to_string(gray_image, lang=tesseract_lang)
            
            # Ejecutar OCR directamente en la imagen PIL
            text_page = pytesseract.image_to_string(image_pil, lang=tesseract_lang)
            cleaned_text_page = text_page.strip()

            ocr_results_per_page.append({
                "page": i + 1,
                "text": cleaned_text_page
            })
            full_ocr_text_parts.append(cleaned_text_page)
            print(f"    Texto página {i+1} (primeros 50 chars): '{cleaned_text_page[:50].replace('\n',' ')}...'")
        
        full_ocr_text = "\n\n".join(full_ocr_text_parts) # Unir con doble salto de línea como en tu API

        print("\n  ✅ Procesamiento OCR local completado.")
        return {
            "texto_completo_ocr": full_ocr_text,
            "detalle_por_pagina": ocr_results_per_page
        }

    except pytesseract.TesseractNotFoundError:
        print("  [ERROR DE TESSERACT] Tesseract no está instalado o no se encuentra en el PATH del sistema.")
        print("  Asegúrate de que la instalación de Tesseract sea correcta y accesible.")
    except Exception as e_pdf2image: # pdf2image puede lanzar varias excepciones
        print(f"  [ERROR EN PDF2IMAGE/POPPLER] Ocurrió un error durante la conversión de PDF a imagen: {e_pdf2image}")
        print(f"  Esto usualmente indica un problema con Poppler (no instalado, no en PATH, o PDF corrupto).")
        import traceback
        traceback.print_exc()
    except Exception as e:
        print(f"  [ERROR INESPERADO] Ocurrió un error durante el procesamiento local: {e}")
        import traceback
        traceback.print_exc()
    return None

In [38]:
# Celda 4: Ejecutar la Función de Procesamiento Local
print("\n" + "="*10 + " PRUEBA DE LÓGICA OCR LOCAL (SIN API) " + "="*10)

if pdf_existe_direct_test:
    # Llamar a la función de procesamiento local
    # Puedes cambiar el DPI o el idioma si lo deseas para probar diferentes configuraciones
    resultado_ocr_local = procesar_pdf_localmente(pdf_file_path_direct_test, dpi_setting=300, tesseract_lang='spa')

    if resultado_ocr_local:
        print("\n--- Resultado del Procesamiento OCR Local ---")
        texto_completo = resultado_ocr_local.get("texto_completo_ocr", "")
        detalle_paginas = resultado_ocr_local.get("detalle_por_pagina", [])
        
        print(f"Texto Completo OCR (primeros 500 caracteres):\n'{texto_completo[:500].replace('\n', '\n  ')}'...")
        
        if detalle_paginas:
            print(f"\nDetalle de las primeras {min(3, len(detalle_paginas))} páginas (de {len(detalle_paginas)} total):")
            for i, page_info in enumerate(detalle_paginas):
                if i < 3:
                     print(f"  Página {page_info.get('page')}: '{page_info.get('text', '')[:100].replace('\n',' ')}...'")
        
        if texto_completo.strip():
            print("\n  ✅ Lógica de OCR parece funcionar: Se extrajo texto.")
        else:
            print("\n  ⚠️ Lógica de OCR: Se completó el proceso, pero no se extrajo texto significativo del PDF.")
    else:
        print("\n  ❌ Falló el procesamiento local del PDF.")
else:
    print("OMITIDO: Prueba de lógica OCR local porque el archivo PDF de prueba no fue encontrado.")

print("\n" + "="*10 + " FIN DE PRUEBA DE LÓGICA OCR LOCAL " + "="*10)


========== PRUEBA DE LÓGICA OCR LOCAL (SIN API) ==========

--- Procesando PDF localmente: c:\repositorios\pdf2Image\NAN_FEHP701932.pdf ---
  Convirtiendo PDF a imágenes (DPI: 300)...
  Se generaron 1 imágenes del PDF.
  Procesando OCR para la página 1...
    Texto página 1 (primeros 50 chars): 'E-S.E. HSOPITAL SAN RAFAEL DE PACHO = MACROPROCESO...'

  ✅ Procesamiento OCR local completado.

--- Resultado del Procesamiento OCR Local ---
Texto Completo OCR (primeros 500 caracteres):
'E-S.E. HSOPITAL SAN RAFAEL DE PACHO
  = MACROPROCESO DE GESTION FINANCIERA FAC-FM-020
  PROCESO DE FACTURACION VERSIÓN
  HOSPITAL FORMATO DE RECIBIDO USUARIO Vv01-2017
  
  Certifico con mi firma que recibí a satisfacción servicios de la siguiente dependencia:
  Fecha de la prestación: 02-05-2025
  
  LABORATORIO CLINICO E 7
  CONSULTA ESPECIALIZADA P X —| ooo _ | — |
  CONSULTA DE MEDICINA GENERAL P |  sewCIORARMEETICO ” | — —|
  PROCEDIMIENTOS MENORES P — | pROMOCIONYMREVENIN — _ | — —|
  
  IMÁGENES DIAG